In [1]:
import wandb

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jon-l-collins. Use `wandb login --relogin` to force relogin


True

In [3]:
from datasets import load_dataset
dataset = load_dataset("yahoo_answers_topics")
dataset

c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset yahoo_answers_topics downloaded and prepared to C:/Users/jonlc/.cache/huggingface/datasets/yahoo_answers_topics/yahoo_answers_topics/1.0.0/0edb353eefe79d9245d7bd7cac5ae6af19530439da520d6dde1c206ee38f4439. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 22.40it/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer'],
        num_rows: 1400000
    })
    test: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer'],
        num_rows: 60000
    })
})

In [4]:
lis = dataset["train"].unique("topic")

In [5]:
lis.sort()
lis

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
dataset = dataset.rename_column("topic", "labels")

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'labels', 'question_title', 'question_content', 'best_answer'],
        num_rows: 1400000
    })
    test: Dataset({
        features: ['id', 'labels', 'question_title', 'question_content', 'best_answer'],
        num_rows: 60000
    })
})

In [10]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(30000))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(10000))

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jonlc\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
small_train_dataset[0]["question_title"]

"My husband's brother was killed at sea in WWII. How do we find out the name of the ship?"

In [13]:
small_train_dataset = small_train_dataset.map(lambda x: tokenizer(x["question_title"], truncation=True, padding="max_length"), batched=True)

In [14]:
#map function which takes textual data (similar to def tokenize function) and gives the input ids and attention mask

small_test_dataset = small_test_dataset.map(lambda x: tokenizer(x["question_title"], truncation=True, padding="max_length"), batched=True)

In [15]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=10)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from wandb.sdk.integration_utils.data_logging import ValidationDataLogger

In [20]:
inputs = small_test_dataset.remove_columns(["input_ids", "attention_mask","labels"])
targets = [small_test_dataset.features["labels"].int2str(x) for x in small_test_dataset["labels"]]

In [ ]:
# small_test_dataset.features["labels"].int2str(2)

In [21]:
val_log = ValidationDataLogger(inputs = inputs[:],
                               targets = targets)

In [27]:
from transformers import Trainer, TrainingArguments

In [24]:
args = TrainingArguments(
    report_to="wandb",
    output_dir="./results",
    learning_rate=5e-5,
    evaluation_strategy="steps",
    max_steps=1000, #overwrites number of epochs
    logging_steps=100,#how frequently results are logged
    eval_steps=500, #how frequently evaluation is done
    run_name="project_a"
    
)    

In [25]:
from datasets import load_metric
import numpy as np
accuracy_metric = load_metric("accuracy")

def compute_metrics(eval):
    predictions, labels = eval
    predictions = np.argmax(predictions, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

C:\Users\jonlc\AppData\Local\Temp\ipykernel_30076\1393689534.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")


In [28]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = small_train_dataset,
    eval_dataset = small_test_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,    
)

In [29]:
trainer.train()

 10%|█         | 101/1000 [00:14<01:53,  7.93it/s]

{'loss': 1.9372, 'learning_rate': 4.5e-05, 'epoch': 0.03}


 20%|██        | 201/1000 [00:28<01:40,  7.96it/s]

{'loss': 1.3621, 'learning_rate': 4e-05, 'epoch': 0.05}


 30%|███       | 301/1000 [00:42<01:30,  7.76it/s]

...

                                                   
100%|██████████| 1000/1000 [04:17<00:00,  8.06it/s]

{'eval_loss': 1.0980162620544434, 'eval_accuracy': 0.6632, 'eval_runtime': 57.977, 'eval_samples_per_second': 172.482, 'eval_steps_per_second': 21.56, 'epoch': 0.27}


100%|██████████| 1000/1000 [04:18<00:00,  3.86it/s]

{'train_runtime': 262.6755, 'train_samples_per_second': 30.456, 'train_steps_per_second': 3.807, 'train_loss': 1.2737845153808594, 'epoch': 0.27}


TrainOutput(global_step=1000, training_loss=1.2737845153808594, metrics={'train_runtime': 262.6755, 'train_samples_per_second': 30.456, 'train_steps_per_second': 3.807, 'train_loss': 1.2737845153808594, 'epoch': 0.27})